In [1]:
import torch
import numpy as np
import pathlib
import dgl
from dgl import backend as F
import torch_geometric
from torch.utils.data import random_split, DataLoader, Dataset
from typing import Dict
from torch import Tensor
from dgl import DGLGraph

In [2]:
#?torch_geometric.nn.pool.ASAPooling

In [3]:
from se3_transformer.model.basis import get_basis, update_basis_with_fused
from se3_transformer.model.transformer import Sequential
from se3_transformer.model.layers.attentiontopK import AttentionBlockSE3
from se3_transformer.model.layers.linear import LinearSE3
from se3_transformer.model.layers.convolution import ConvSE3, ConvSE3FuseLevel
from se3_transformer.model.layers.norm import NormSE3
from se3_transformer.model.layers.pooling import GPooling
from se3_transformer.runtime.utils import str2bool, to_cuda
from se3_transformer.model.fiber import Fiber
from se3_transformer.model.transformer import get_populated_edge_features

In [4]:
torch.cuda.is_available()

True

In [5]:
data_path_str  = 'data/h4_ca_coords.npz'
test_limit = 128
rr = np.load(data_path_str)
ca_coords = [rr[f] for f in rr.files][0][:test_limit,:,:3]
ca_coords.shape

(128, 65, 3)

In [6]:
#goal define edges of
#connected backbone 1, 
#unconnected atoms 0,


def get_midpoint(ep_in):
    """Get midpoint, of each batched set of points"""
    
    #calculate midpoint
    midpoint = ep_in.sum(axis=1)/np.repeat(ep_in.shape[1], ep_in.shape[2])
    
    return midpoint


def normalize_points(input_xyz, print_dist=False):
    
    #broadcast to distance matrix [Batch, M, R3] to [Batch,M,1, R3] to [Batch,1,M, R3] to [Batch, M,M, R3] 
    vec_diff = input_xyz[...,None,:]-input_xyz[...,None,:,:]
    dist = np.sqrt(np.sum(np.square(vec_diff),axis=len(input_xyz.shape)))
    furthest_dist = np.max(dist)
    centroid  = get_midpoint(input_xyz)
    if print_dist:
        print(f'largest distance {furthest_dist:0.1f}')
    
    xyz_mean_zero = input_xyz - centroid[:,None,:]
    return xyz_mean_zero/furthest_dist



def define_graph_edges(n_nodes):
    #connected backbone

    con_v1 = np.arange(n_nodes-1) #vertex 1 of edges in chronological order
    con_v2 = np.arange(1,n_nodes) #vertex 2 of edges in chronological order

    ind = con_v1*(n_nodes-1)+con_v2-1 #account for removed self connections (-1)


    #unconnected backbone

    nodes = np.arange(n_nodes)
    v1 = np.repeat(nodes,n_nodes-1) #starting vertices, same number repeated for each edge

    start_v2 = np.repeat(np.arange(n_nodes)[None,:],n_nodes,axis=0)
    diag_ind = np.diag_indices(n_nodes)
    start_v2[diag_ind] = -1 #diagonal of matrix is self connections which we remove (self connections are managed elsewhere)
    v2 = start_v2[start_v2>-0.5] #remove diagonal and flatten

    edge_data = torch.zeros(len(v2))
    edge_data[ind] = 1
    
    return v1,v2,edge_data, ind

def make_pe_encoding(n_nodes=65, embed_dim = 12, scale = 1000, cast_type=torch.float32, print_out=False):
    #positional encoding of node
    i_array = np.arange(1,(embed_dim/2)+1)
    wk = (1/(scale**(i_array*2/embed_dim)))
    t_array = np.arange(n_nodes)
    si = torch.tensor(np.sin(wk*t_array.reshape((-1,1))))
    ci = torch.tensor(np.cos(wk*t_array.reshape((-1,1))))
    pe = torch.stack((si,ci),axis=2).reshape(t_array.shape[0],embed_dim).type(cast_type)
    
    if print_out == True:
        for x in range(int(n_nodes/12)):
            print(np.round(pe[x],1))
    
    return pe
    
    
#v1,v2,edge_data, ind = define_graph_edges(n_nodes)
norm_p = normalize_points(ca_coords,print_dist=True)
pe = make_pe_encoding(n_nodes=65, embed_dim = 12, scale = 10, print_out=True)

largest distance 32.8
tensor([0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.])
tensor([0.6000, 0.8000, 0.4000, 0.9000, 0.3000, 1.0000, 0.2000, 1.0000, 0.1000,
        1.0000, 0.1000, 1.0000])
tensor([1.0000, 0.2000, 0.8000, 0.6000, 0.6000, 0.8000, 0.4000, 0.9000, 0.3000,
        1.0000, 0.2000, 1.0000])
tensor([ 0.9000, -0.5000,  1.0000,  0.2000,  0.8000,  0.6000,  0.6000,  0.8000,
         0.4000,  0.9000,  0.3000,  1.0000])
tensor([ 0.4000, -0.9000,  1.0000, -0.3000,  1.0000,  0.3000,  0.8000,  0.7000,
         0.6000,  0.8000,  0.4000,  0.9000])


In [7]:
#v1,v2,edge_data, ind = define_graph_edges(4)

In [8]:
#?dgl.nn.pytorch.KNNGraph, nearest neighbor graph maker
def define_graph(batch_size=8,n_nodes=65):
    
    v1,v2,edge_data, ind = define_graph_edges(n_nodes)
    pe = make_pe_encoding(n_nodes=65)
    
    graphList = []
    
    for i in range(batch_size):
        
        g = dgl.graph((v1,v2))
        g.edata['con'] = edge_data
        g.ndata['pe'] = pe

        graphList.append(g)
        
    batched_graph = dgl.batch(graphList)

    return batched_graph

In [9]:
class Graph_4H_Dataset(Dataset):
    def __init__(self, ca_coordinates, limit=1000, cast_type=torch.float32):
        
        self.ca_coords = ca_coordinates
        self.norm_ca = normalize_points(ca_coordinates)
        
        n_nodes = self.ca_coords.shape[1] 
        
        v1,v2,edge_data, ind = define_graph_edges(n_nodes)
        pe = make_pe_encoding(n_nodes=n_nodes)

        graphList = []

        for i,c in enumerate(self.norm_ca):

            g = dgl.graph((v1,v2))
            g.edata['con'] = edge_data.type(cast_type).reshape((-1,1))
            g.ndata['pe'] = pe
            g.ndata['pos'] = torch.tensor(c,dtype=torch.float32)

            graphList.append(g)
        
        self.graphList = graphList


    def __len__(self):
        return len(self.graphList)

    def __getitem__(self, idx):
        return self.graphList[idx]

def _get_relative_pos(graph_in: dgl.DGLGraph) -> torch.Tensor:
    x = graph_in.ndata['pos']
    src, dst = graph_in.edges()
    rel_pos = x[dst] - x[src]
    return rel_pos
    
#needs to be done
class H4_DataModule():
    """
    Datamodule wrapping hGen data set. 8 Helical endpoints defining a four helix protein.
    """
    #8 long positional encoding
    NODE_FEATURE_DIM = 12
    EDGE_FEATURE_DIM = 1 # 0 or 1 helix or loop

    def __init__(self,
                 ca_coords: np.array, batch_size=8):
        
        self.GraphDatasetObj = Graph_4H_Dataset(ca_coords)
        self.gds = DataLoader(self.GraphDatasetObj, batch_size=batch_size, shuffle=True, drop_last=True,
                              collate_fn=self._collate)
        
    
        
    def _collate(self, graphs):
        batched_graph = dgl.batch(graphs)
        #reshape that batched graph to redivide into the individual graphs
        edge_feats = {'0': batched_graph.edata['con'][:, :self.EDGE_FEATURE_DIM, None]}
        batched_graph.edata['rel_pos'] = _get_relative_pos(batched_graph)
        # get node features
        node_feats = {'0': batched_graph.ndata['pe'][:, :self.NODE_FEATURE_DIM, None]}
        
        return (batched_graph, node_feats, edge_feats)

In [10]:
dm = H4_DataModule(ca_coords)

In [11]:
n_nodes = 65
NODE_FEATURE_DIM = 12
EDGE_FEATURE_DIM = 1 # probably expand to [2] one hot primary connect 
num_degrees = 4 # how many levels of spherical harmonics to use
num_channels = 8 # how many
num_heads = 4
channels_div = 2
max_degree = 4

use_layer_norm = False
fuse_level = ConvSE3FuseLevel.NONE


fiber_in=Fiber({0: NODE_FEATURE_DIM})
fiber_hidden=Fiber({0: num_degrees * num_channels})
fiber_edge=Fiber({0: EDGE_FEATURE_DIM})
fiber_out = Fiber({0: num_degrees * num_channels}) # can this be arbitrary, or projected

In [12]:
ablock = AttentionBlockSE3(fiber_in=fiber_in,
               fiber_out=fiber_hidden,
               fiber_edge=fiber_edge,
               num_heads=num_heads,
               channels_div=channels_div,
               use_layer_norm=use_layer_norm,
               max_degree=max_degree,
               fuse_level=fuse_level,
               low_memory='True')
acuda = ablock.to('cuda')

In [66]:
def topK_se3(graph, feat, xi, k):
    #remove this read from graph code, since se3 transformer natively uses pulled out feats from graph
    # READOUT_ON_ATTRS = {
    #     "nodes": ("ndata", "batch_num_nodes", "number_of_nodes"),
    #     "edges": ("edata", "batch_num_edges", "number_of_edges"),
    # }
    # _, batch_num_objs_attr, _ = READOUT_ON_ATTRS["nodes"]

    # #this is a fancy way of saying 'batch_num_nodes
    # data = getattr(bg, "nodes")[None].data
    # if F.ndim(data[feat]) > 2:
    #     raise DGLError(
    #         "Only support {} feature `{}` with dimension less than or"
    #         " equal to 2".format(typestr, feat)
    #     )
    # feat = data[feat]


    hidden_size = feat.shape[-1]
    batch_num_objs = getattr(bg, 'batch_num_nodes')(None)
    batch_size = len(batch_num_objs)
    descending = True

    length = max(max(F.asnumpy(batch_num_objs)), k) #max k or batch of nodes size
    fill_val = -float("inf") if descending else float("inf")
    
    feat_y = F.pad_packed_tensor(
        feat, batch_num_objs, fill_val, l_min=k
    )  # (batch_size, l, d)

    order = F.argsort(feat_y, 1, descending=descending)
    topk_indices = F.slice_axis(order, 1, 0, k)

    #get batch shifts
    feat_ = F.reshape(feat_y, (-1,))
    shift = F.repeat(
        F.arange(0, batch_size), k * hidden_size, -1
    ) * length * hidden_size + F.cat(
        [F.arange(0, hidden_size)] * batch_size * k, -1
    )
    
    shift = F.copy_to(shift, F.context(feat))
    topk_indices_ = F.reshape(topk_indices, (-1,)) * hidden_size + shift
    #trainable params gather
    out_y = F.reshape(F.gather_row(feat_, topk_indices_), (batch_size*k, -1))
    out_y = F.replace_inf_with_zero(out_y)
    #nodes features gather
    out_xi = F.reshape(F.gather_row(xi, topk_indices_), (batch_size*k, -1))
    out_xi = F.replace_inf_with_zero(out_xi)
    return out_y, out_xi, topk_indices_

In [83]:

class TopK_Pool(torch.nn.Module):
    """
    https://arxiv.org/pdf/1905.05178.pdf
    Project Node Features to 1D for topK pooling using trainable weights
    Only type '0' features coded, would need to mix features between degrees, avoiding this for now
    """

    def __init__(self, fiber_in: Fiber, k=5):
        super().__init__()
        self.k = k
        fiber_out = Fiber({0: 1}) #convert to 1D of nodes
        self.weights = torch.nn.ParameterDict({
            str(degree_out): torch.nn.Parameter(
                torch.randn(channels_out, fiber_in[degree_out]) / np.sqrt(fiber_in[degree_out]))
            for degree_out, channels_out in fiber_out
        })
        


    def forward(self, features: Dict[str, Tensor], graph: DGLGraph) -> Dict[str, Tensor]:
        #add topK selection, sigmoid, return nodes
        yi = {
            degree: torch.div(self.weights[degree] @ features[degree], self.weights[degree].norm())
            for degree, weight in self.weights.items()
        }
        
        y_selected, feats_selected, topk_indices_batched = topK_se3(graph, yi['0'], features['0'], self.k)
        return torch.sigmoid(y_selected)*feats_selected, topk_indices_batched
        
        
        
        
        

In [84]:
tk = TopK_Pool(fiber_hidden)
tk_cuda = tk.to('cuda')
# tblock = [ablock,tk]
# model = Sequential(*tblock)

In [85]:
for i, inp in enumerate(dm.gds):
    batched_graph, node_feats, edge_feats = inp
    break
    
bg = to_cuda(batched_graph)
edge_feats = to_cuda(edge_feats)
node_feats = to_cuda(node_feats)

basis = get_basis(bg.edata['rel_pos'], max_degree=max_degree,
                                   compute_gradients=False,
                                   use_pad_trick=False)
#concatenate on the distances of the edge based on 'rel pos'
edge_feats_cat = get_populated_edge_features(bg.edata['rel_pos'], edge_feats)

In [86]:
out = acuda.forward(node_feats, edge_feats_cat,graph=bg,basis=basis)

In [88]:
out, inde= tk.forward(out, bg)

In [89]:
out.shape

torch.Size([40, 32])

In [73]:
kc.shape

torch.Size([40, 32])

In [82]:
(kc*torch.sigmoid(yi)).shape

torch.Size([40, 32])

In [74]:
m.shape

torch.Size([40])

In [49]:
k.shape

torch.Size([40])

In [42]:
out['0'].shape

torch.Size([520, 32, 1])

In [43]:
out_y, out_xi, topk_indices_ = topK_se3(bg, yi['0'], out['0'], k)

In [33]:
feat_xi.shape

torch.Size([8, 65, 32, 1])

In [34]:
topk_indices

tensor([ 54,  53,  11,  12,  55, 119, 118,  76,  77, 120, 184, 183, 141, 185,
        142, 249, 248, 206, 250, 207, 314, 313, 271, 315, 272, 379, 378, 336,
        337, 380, 444, 443, 401, 445, 402, 509, 508, 466, 467, 510],
       device='cuda:0')

In [22]:
feat = yi['0']
hidden_size = feat.shape[-1]
xi = out['0']


In [23]:
batch_num_objs = getattr(bg, 'batch_num_nodes')(None)
batch_num_objs

tensor([65, 65, 65, 65, 65, 65, 65, 65], device='cuda:0')

In [24]:
batch_size = len(batch_num_objs)
batch_size

8

In [25]:
max(max(F.asnumpy(batch_num_objs)),k)

65

In [26]:
length = max(max(F.asnumpy(batch_num_objs)), k)
length

65

In [27]:
descending = True
fill_val = -float("inf") if descending else float("inf")
fill_val

-inf

In [30]:
feat_y = F.pad_packed_tensor(
        feat, batch_num_objs, fill_val, l_min=k
    )  # (batch_size, l, d)
feat_xi = F.pad_packed_tensor(
        xi, batch_num_objs, fill_val, l_min=k
    )  # (batch_size, l, d)

In [31]:
feat_xi.shape

torch.Size([8, 65, 32, 1])

In [32]:
order = F.argsort(feat_y, 1, descending=descending)
topk_indices = F.slice_axis(order, 1, 0, k)


In [34]:
topk_indices

tensor([[[[12]],

         [[11]],

         [[13]],

         [[10]],

         [[14]]],


        [[[12]],

         [[11]],

         [[13]],

         [[10]],

         [[14]]],


        [[[12]],

         [[11]],

         [[13]],

         [[10]],

         [[14]]],


        [[[12]],

         [[11]],

         [[13]],

         [[10]],

         [[14]]],


        [[[12]],

         [[11]],

         [[13]],

         [[10]],

         [[14]]],


        [[[12]],

         [[11]],

         [[13]],

         [[10]],

         [[14]]],


        [[[12]],

         [[11]],

         [[13]],

         [[10]],

         [[14]]],


        [[[12]],

         [[11]],

         [[13]],

         [[10]],

         [[14]]]], device='cuda:0')

In [33]:
feat_ = F.reshape(feat_y, (-1,))
feat_.shape

torch.Size([520])

In [108]:
 hidden_size_xi

tensor(32, device='cuda:0')

In [113]:
shift = F.repeat(
        F.arange(0, batch_size), k * hidden_size, -1
    ) * length * hidden_size + F.cat(
        [F.arange(0, hidden_size)] * batch_size * k, -1
    )
shift = F.copy_to(shift, F.context(feat))




In [146]:
topk_indices_ = F.reshape(topk_indices, (-1,)) * hidden_size + shift

In [150]:
out2 = F.reshape(F.gather_row(feat_2, topk_indices_), (batch_size, k, -1))

In [ ]:
hidden_size = feat.shape[-1]
    batch_num_objs = getattr(bg, 'batch_num_nodes')(None)
    batch_size = len(batch_num_objs)
    descending = True

    length = max(max(F.asnumpy(batch_num_objs)), k) #get maximum nodes in batch, or k
    fill_val = -float("inf") if descending else float("inf")
    feat_ = F.pad_packed_tensor(
        feat, batch_num_objs, fill_val, l_min=k
    )  # (batch_size, l, d)
    
    #use argsort to grab top nodes based on feat_ for each batch
    order = F.argsort(feat_, 1, descending=descending)
    topk_indices = F.slice_axis(order, 1, 0, k)


    feat_ = F.reshape(feat_, (-1,))
    #create batch based shift for each indices
    shift = F.repeat(
        F.arange(0, batch_size), k * hidden_size, -1
    ) * length * hidden_size + F.cat(
        [F.arange(0, hidden_size)] * batch_size * k, -1
    )
    #take new indi
    shift = F.copy_to(shift, F.context(feat))
    #use batch shift and hidden size to get the approriate indices
    topk_indices_ = F.reshape(topk_indices, (-1,)) * hidden_size + shift

    out = F.reshape(F.gather_row(feat_, topk_indices_), (batch_size, k, -1))
    out = F.replace_inf_with_zero(out)
    return out, topk_indices

In [ ]:
out = F.reshape(F.gather_row(feat_, topk_indices_), (batch_size, k, -1))

In [141]:
?F.gather_row

In [144]:
feat_2 = out['0'].squeeze()
feat_2.shape

torch.Size([520, 32])

In [147]:
topk_indices_

tensor([  0,   1,   2,   3,   4,  65,  66,  67,  68,  69, 130, 131, 132, 133,
        134, 195, 196, 197, 198, 199, 260, 261, 262, 263, 264, 325, 326, 327,
        328, 329, 390, 391, 392, 393, 394, 455, 456, 457, 458, 459],
       device='cuda:0')

In [149]:
F.gather_row(feat_2, topk_indices_).shape

torch.Size([40, 32])

In [24]:
topkInd.squeeze()

tensor([[48, 49, 47, 50, 46],
        [48, 49, 47, 50, 46],
        [48, 49, 47, 50, 46],
        [48, 49, 47, 50, 46],
        [48, 49, 47, 50, 46],
        [48, 49, 47, 50, 46],
        [48, 49, 47, 50, 46],
        [48, 49, 47, 50, 46]], device='cuda:0')

In [114]:
model = model.to('cuda')

In [119]:
edge_feats['0'].shape

torch.Size([33280, 1, 1])

In [120]:
edge_feats_cat['0'][0]

tensor([[1.0000],
        [0.1159]], device='cuda:0')

In [121]:
acuda.forward(node_feats, edge_feats_cat,graph=bg,basis=basis)

In [122]:
fib = model(node_feats, edge_feats_cat, graph=bg, basis=basis)

TypeError: forward() got multiple values for argument 'graph'

In [87]:
fib['0'].shape

torch.Size([520, 1, 1])

In [42]:
520/8

65.0

In [111]:
proj_z, z_concat, z, edge_w = fib

In [112]:
edge_w.shape

torch.Size([33280, 4, 1, 1])

In [113]:
z['0'].shape

torch.Size([520, 16, 1])

In [114]:
z_concat['0'].shape

torch.Size([520, 28, 1])

In [115]:
proj_z['0'].shape

torch.Size([520, 32, 1])

In [116]:
proj_z['0'].squeeze(-1).shape

torch.Size([520, 32])

In [117]:
520/8

65.0

In [118]:
bg.ndata['h'] = proj_z['0'].squeeze(-1)

In [119]:
?dgl.topk_nodes

In [120]:
feat = proj_z['0'].squeeze(-1)

In [125]:
feat[66][0] = 10

In [126]:
feat[66][0]

tensor(10., device='cuda:0', grad_fn=<SelectBackward0>)

In [127]:
out, topk_indices = topK_se3(bg, feat, 5)

In [128]:
topk_indices

tensor([[[64, 12,  3,  ..., 47, 57, 39],
         [63, 11,  4,  ..., 48, 56, 38],
         [62, 13,  2,  ..., 46, 58, 40],
         [61, 10,  5,  ..., 49, 55, 37],
         [60, 14,  1,  ..., 45, 59, 41]],

        [[ 1, 12,  3,  ..., 47, 57, 39],
         [64, 11,  4,  ..., 48, 56, 38],
         [63, 13,  2,  ..., 46, 58, 40],
         [62, 10,  5,  ..., 49, 55, 37],
         [61, 14,  1,  ..., 45, 59, 41]],

        [[64, 12,  3,  ..., 47, 57, 39],
         [63, 11,  4,  ..., 48, 56, 38],
         [62, 13,  2,  ..., 46, 58, 40],
         [61, 10,  5,  ..., 49, 55, 37],
         [60, 14,  1,  ..., 45, 59, 41]],

        ...,

        [[64, 12,  3,  ..., 47, 57, 39],
         [63, 11,  4,  ..., 48, 56, 38],
         [62, 13,  2,  ..., 46, 58, 40],
         [61, 10,  5,  ..., 49, 55, 37],
         [60, 14,  1,  ..., 45, 59, 41]],

        [[64, 12,  3,  ..., 47, 57, 39],
         [63, 11,  4,  ..., 48, 56, 38],
         [62, 13,  2,  ..., 46, 58, 40],
         [61, 10,  5,  ..., 49, 55,

In [105]:
topk_indices.shape

torch.Size([8, 5, 32])

In [96]:
#feat = proj_z['0'].squeeze(-1)
def topK_se3(graph, feat, k):
    #remove this read from graph code, since se3 transformer natively uses pulled out feats from graph
    # READOUT_ON_ATTRS = {
    #     "nodes": ("ndata", "batch_num_nodes", "number_of_nodes"),
    #     "edges": ("edata", "batch_num_edges", "number_of_edges"),
    # }
    # _, batch_num_objs_attr, _ = READOUT_ON_ATTRS["nodes"]

    # #this is a fancy way of saying 'batch_num_nodes
    # data = getattr(bg, "nodes")[None].data
    # if F.ndim(data[feat]) > 2:
    #     raise DGLError(
    #         "Only support {} feature `{}` with dimension less than or"
    #         " equal to 2".format(typestr, feat)
    #     )
    # feat = data[feat]


    hidden_size = feat.shape[-1]
    batch_num_objs = getattr(bg, 'batch_num_nodes')(None)
    batch_size = len(batch_num_objs)
    descending = True

    length = max(max(F.asnumpy(batch_num_objs)), k)
    fill_val = -float("inf") if descending else float("inf")
    feat_ = F.pad_packed_tensor(
        feat, batch_num_objs, fill_val, l_min=k
    )  # (batch_size, l, d)
    
    order = F.argsort(feat_, 1, descending=descending)
    topk_indices = F.slice_axis(order, 1, 0, k)


    feat_ = F.reshape(feat_, (-1,))
    shift = F.repeat(
        F.arange(0, batch_size), k * hidden_size, -1
    ) * length * hidden_size + F.cat(
        [F.arange(0, hidden_size)] * batch_size * k, -1
    )
    shift = F.copy_to(shift, F.context(feat))
    topk_indices_ = F.reshape(topk_indices, (-1,)) * hidden_size + shift
    
    out = F.reshape(F.gather_row(feat_, topk_indices_), (batch_size, k, -1))
    out = F.replace_inf_with_zero(out)
    return out, topk_indices

In [43]:
?F.argsort

In [40]:
order = F.argsort(feat_, 1, descending=descending)
topk_indices = F.slice_axis(order, 1, 0, k)


feat_ = F.reshape(feat_, (-1,))
shift = F.repeat(
    F.arange(0, batch_size), k * hidden_size, -1
) * length * hidden_size + F.cat(
    [F.arange(0, hidden_size)] * batch_size * k, -1
)
shift = F.copy_to(shift, F.context(feat))
topk_indices_ = F.reshape(topk_indices, (-1,)) * hidden_size + shift

In [41]:
topk_indices

tensor([[[64, 12,  3,  ..., 47, 57, 39],
         [63, 11,  4,  ..., 48, 56, 38],
         [62, 13,  2,  ..., 46, 58, 40],
         ...,
         [44,  8,  7,  ..., 51, 38,  1],
         [59, 16,  8,  ..., 43, 39, 42],
         [47, 17,  9,  ..., 52, 37, 35]],

        [[64, 12,  3,  ..., 47, 57, 39],
         [63, 11,  4,  ..., 48, 56, 38],
         [62, 13,  2,  ..., 46, 58, 40],
         ...,
         [44,  8,  7,  ..., 51, 38,  1],
         [59, 16,  8,  ..., 43, 53, 42],
         [47, 17,  9,  ..., 52, 37, 35]],

        [[64, 12,  3,  ..., 47, 57, 39],
         [63, 11,  4,  ..., 48, 56, 38],
         [62, 13,  2,  ..., 46, 58, 40],
         ...,
         [44,  8,  7,  ..., 51, 38,  1],
         [59, 16,  8,  ..., 43, 37, 42],
         [47, 17,  9,  ..., 52, 39, 35]],

        ...,

        [[64, 12,  3,  ..., 47, 57, 39],
         [63, 11,  4,  ..., 48, 56, 38],
         [62, 13,  2,  ..., 46, 58, 40],
         ...,
         [44,  8,  7,  ..., 51, 38,  1],
         [59, 16,  8, 

In [80]:
data

{'pe': tensor([[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  ...,  1.0000e+00,
          0.0000e+00,  1.0000e+00],
        [ 3.1098e-01,  9.5042e-01,  9.9833e-02,  ...,  9.9999e-01,
          1.0000e-03,  1.0000e+00],
        [ 5.9113e-01,  8.0658e-01,  1.9867e-01,  ...,  9.9998e-01,
          2.0000e-03,  1.0000e+00],
        ...,
        [ 6.8643e-01,  7.2720e-01, -8.3089e-02,  ...,  9.8084e-01,
          6.1960e-02,  9.9808e-01],
        [ 8.7854e-01,  4.7767e-01,  1.6814e-02,  ...,  9.8022e-01,
          6.2958e-02,  9.9802e-01],
        [ 9.8352e-01,  1.8078e-01,  1.1655e-01,  ...,  9.7959e-01,
          6.3956e-02,  9.9795e-01]], device='cuda:0'), 'pos': tensor([[-0.3364, -0.0208, -0.0076],
        [-0.3327,  0.0943, -0.0212],
        [-0.3273,  0.1009,  0.0944],
        ...,
        [-0.0216,  0.2502,  0.1192],
        [-0.0466,  0.2097,  0.0135],
        [-0.1549,  0.2505,  0.0062]], device='cuda:0'), 'h': tensor([[ 0.4256, -0.0988, -0.1517,  ...,  0.4718,  0.2854, -0.3482],
       

In [ ]:
#remake without pulling from graph?

READOUT_ON_ATTRS = {
    "nodes": ("ndata", "batch_num_nodes", "number_of_nodes"),
    "edges": ("edata", "batch_num_edges", "number_of_edges"),
}

def _topk_on(graph, typestr, feat, k, descending, sortby, ntype_or_etype):
    """Internal function to take graph-wise top-k node/edge features of
    field :attr:`feat` in :attr:`graph` ranked by keys at given
    index :attr:`sortby`. If :attr:`descending` is set to False, return the
    k smallest elements instead.

    Parameters
    ---------
    graph : DGLGraph
        The graph
    typestr : str
        'nodes' or 'edges'
    feat : str
        The feature field name.
    k : int
        The :math:`k` in "top-:math`k`".
    descending : bool
        Controls whether to return the largest or smallest elements,
         defaults to True.
    sortby : int
        The key index we sort :attr:`feat` on, if set to None, we sort
        the whole :attr:`feat`.
    ntype_or_etype : str, tuple of str
        Node/edge type.

    Returns
    -------
    sorted_feat : Tensor
        A tensor with shape :math:`(B, K, D)`, where
        :math:`B` is the batch size of the input graph.
    sorted_idx : Tensor
        A tensor with shape :math:`(B, K)`(:math:`(B, K, D)` if sortby
        is set to None), where
        :math:`B` is the batch size of the input graph, :math:`D`
        is the feature size.


    Notes
    -----
    If an example has :math:`n` nodes/edges and :math:`n<k`, in the first
    returned tensor the :math:`n+1` to :math:`k`th rows would be padded
    with all zero; in the second returned tensor, the behavior of :math:`n+1`
    to :math:`k`th elements is not defined.
    """
    _, batch_num_objs_attr, _ = READOUT_ON_ATTRS[typestr]
    data = getattr(graph, typestr)[ntype_or_etype].data
    if F.ndim(data[feat]) > 2:
        raise DGLError(
            "Only support {} feature `{}` with dimension less than or"
            " equal to 2".format(typestr, feat)
        )
    feat = data[feat]
    hidden_size = F.shape(feat)[-1]
    batch_num_objs = getattr(graph, batch_num_objs_attr)(ntype_or_etype)
    batch_size = len(batch_num_objs)
    length = max(max(F.asnumpy(batch_num_objs)), k)
    fill_val = -float("inf") if descending else float("inf")
    feat_ = F.pad_packed_tensor(
        feat, batch_num_objs, fill_val, l_min=k
    )  # (batch_size, l, d)

    if F.backend_name == "pytorch" and sortby is not None:
        # PyTorch's implementation of top-K
        keys = feat_[..., sortby]  # (batch_size, l)
        return _topk_torch(keys, k, descending, feat_)
    else:
        # Fallback to framework-agnostic implementation of top-K
        if sortby is not None:
            keys = F.squeeze(F.slice_axis(feat_, -1, sortby, sortby + 1), -1)
            order = F.argsort(keys, -1, descending=descending)
        else:
            order = F.argsort(feat_, 1, descending=descending)
        topk_indices = F.slice_axis(order, 1, 0, k)

        if sortby is not None:
            feat_ = F.reshape(feat_, (batch_size * length, -1))
            shift = F.repeat(F.arange(0, batch_size) * length, k, -1)
            shift = F.copy_to(shift, F.context(feat))
            topk_indices_ = F.reshape(topk_indices, (-1,)) + shift
        else:
            feat_ = F.reshape(feat_, (-1,))
            shift = F.repeat(
                F.arange(0, batch_size), k * hidden_size, -1
            ) * length * hidden_size + F.cat(
                [F.arange(0, hidden_size)] * batch_size * k, -1
            )
            shift = F.copy_to(shift, F.context(feat))
            topk_indices_ = F.reshape(topk_indices, (-1,)) * hidden_size + shift
        out = F.reshape(F.gather_row(feat_, topk_indices_), (batch_size, k, -1))
        out = F.replace_inf_with_zero(out)
        return out, topk_indices

In [73]:
for x in range(8):
    for y in range(30):
        if len(np.unique(sort_ind[x][:,y].cpu().numpy())) != 30:
            print(False)

In [56]:
sort_feat[0]

tensor([[ 1.2111, -0.0900,  0.7318,  0.2344, -0.0669,  0.0927,  0.5824,  0.4390,
          0.0690,  0.5615,  0.7202,  0.1591,  0.5839,  0.1566,  0.3246,  0.1450,
          0.8620,  0.5528,  1.3103,  0.5718,  0.4452, -0.5404,  0.4007,  0.3480,
          0.4214,  0.7397,  0.1821,  0.0995,  0.1473,  0.8472,  0.4826,  0.7203],
        [ 1.2044, -0.0912,  0.7244,  0.2296, -0.1206,  0.0902,  0.5798,  0.4309,
          0.0607,  0.5256,  0.7131,  0.1099,  0.5822,  0.1490,  0.3212,  0.1406,
          0.8589,  0.5515,  1.3042,  0.5710,  0.4410, -0.5436,  0.3697,  0.3449,
          0.4145,  0.7275,  0.1739,  0.0936,  0.1362,  0.8349,  0.4770,  0.7097],
        [ 1.2012, -0.0977,  0.7028,  0.2276, -0.1264,  0.0872,  0.5759,  0.4240,
          0.0556,  0.5194,  0.6933,  0.0538,  0.5749,  0.1320,  0.3177,  0.1326,
          0.8483,  0.5320,  1.2934,  0.5513,  0.4327, -0.5531,  0.3374,  0.3360,
          0.4096,  0.7245,  0.1727,  0.0884,  0.1138,  0.8332,  0.4558,  0.6915],
        [ 1.1841, -0.0988

In [54]:
sort_ind[0].shape

torch.Size([10, 32])

In [ ]:
model = SE3TransformerPooled(
        fiber_in=Fiber({0: dm.NODE_FEATURE_DIM}),
        fiber_out=Fiber({0: num_degrees * num_channels}),
        fiber_edge=Fiber({0: dm.EDGE_FEATURE_DIM}),
        output_dim=1,
        tensor_cores=using_tensor_cores(False),
        num_degrees=num_degrees,
        num_channels=num_channels,
        **kwargs
    )

In [41]:
?dgl.nn.pytorch.

In [ ]:
# get dataset from pdb using npose utils
# import util.npose_util as nu
# import os
# import numpy as np

# model_direc = '/mnt/c/Users/nwood/OneDrive/Desktop/hTest/HelixGen_master/data/4H_dataset/models/'

# fL = os.listdir(model_direc)
# coords = np.zeros((len(fL),65*5,4)) #65 aa, 5 atoms per aa
# for i,file in enumerate(fL):
#     coords[i] = nu.npose_from_file(f'{model_direc}/{file}')

# coords_out = coords.reshape((27894,65,5,4))[...,:3]
# ca_coords = coords_out.reshape((27894,65,5,3))[:,:,1,:]

# np.savez_compressed('../gudiff/data/h4_coords.npz',coords_out)
# np.savez_compressed('../gudiff/data/h4_ca_coords.npz',ca_coords)

In [10]:




def normalize_pc(points):
    """Center at Zero Divide furtherst points"""
    centroid = np.mean(points, axis=0)
    points -= centroid
    #since the points are centered zero, the furthest points is the abs value di
    furthest_distance = np.max(np.sqrt(np.sum(abs(points)**2,axis=-1)))
    points /= furthest_distance

    return points, furthest_distance
    
def make_pe_encoding(i_pos=8, embed_dim = 8, scale = 10, cast_type=torch.float32):
    #positional encoding of node
    i_array = np.arange(1,(embed_dim/2)+1)
    wk = (1/(scale**(i_array*2/embed_dim)))
    t_array = np.arange(i_pos)
    si = torch.tensor(np.sin(wk*t_array.reshape((-1,1))))
    ci = torch.tensor(np.cos(wk*t_array.reshape((-1,1))))
    pe = torch.stack((si,ci),axis=2).reshape(t_array.shape[0],embed_dim).type(cast_type)
    return pe


def make_graph_struct(batch_size=32, n_nodes = 8):
    # make a fake graph to be filled with generator outputs
    
    v1 = np.arange(n_nodes-1) #vertex 1 of edges in chronological order
    v2 = np.arange(1,n_nodes) #vertex 2 of edges in chronological order

    ss = np.zeros(len(v1),dtype=np.int32)
    ss[np.arange(ss.shape[0])%2==0]=1  #alternate 0,1 for helix, loop, helix, etc
    ss = ss[:,None] #unsqueeze
    
    pe = make_pe_encoding(i_pos=8, embed_dim = 8, scale = 10, cast_type=torch.float32)

    graphList = []
    for i in range(batch_size):
        g = dgl.graph((v1,v2))
        g.edata['ss'] = torch.tensor(ss,dtype=torch.float32)
        g.ndata['pe'] = pe

        graphList.append(g)

    batched_graph = dgl.batch(graphList)

    return batched_graph


class GraphDataset(Dataset):
    def __init__(self, ep_file : pathlib.Path, limit=1000):
        self.data_path = ep_file
        rr = np.load(self.data_path)
        ep = [rr[f] for f in rr.files][0][:1000]
        
        #need to save furthest distance to regen later
        #maybe consider small change for next steps
        ep, self.furthest_distance = normalize_pc(ep.reshape((-1,3)))
        self.ep = ep.reshape((-1,8,3))
        
        
        v1 = np.arange(self.ep.shape[1]-1) #vertex 1 of edges in chronological order
        v2 = np.arange(1,self.ep.shape[1]) #vertex 2 of edges in chronological order

        ss = np.zeros(len(v1))
        ss[np.arange(ss.shape[0])%2==0]=1  #alternate 0,1 for helix, loop, helix, etc
        ss = ss[:,None] #unsqueeze

        #positional encoding of node
        pe = make_pe_encoding(i_pos=8, embed_dim = 8, scale = 10, cast_type=torch.float32)

        graphList = []

        for i,c in enumerate(self.ep):

            g = dgl.graph((v1,v2))
            g.ndata['pos'] = torch.tensor(c,dtype=torch.float32)
            g.edata['ss'] = torch.tensor(ss,dtype=torch.float32)
            g.ndata['pe'] = pe

            graphList.append(g)
        
        self.graphList = graphList


    def __len__(self):
        return len(self.graphList)

    def __getitem__(self, idx):
        return self.graphList[idx]

    
class HGenDataModule():
    """
    Datamodule wrapping hGen data set. 8 Helical endpoints defining a four helix protein.
    """
    #8 long positional encoding
    NODE_FEATURE_DIM = 8
    EDGE_FEATURE_DIM = 1 # 0 or 1 helix or loop

    def __init__(self,
                 data_dir: pathlib.Path, batch_size=32):
        
        self.data_dir = data_dir 
        self.GraphDatasetObj = GraphDataset(self.data_dir)
        self.gds = DataLoader(self.GraphDatasetObj,batch_size=batch_size, shuffle=True, drop_last=True,
                              collate_fn=self._collate)
        
    
        
    def _collate(self, graphs):
        batched_graph = dgl.batch(graphs)
        #reshape that batched graph to redivide into the individual graphs
        edge_feats = {'0': batched_graph.edata['ss'][:, :self.EDGE_FEATURE_DIM, None]}
        batched_graph.edata['rel_pos'] = _get_relative_pos(batched_graph)
        # get node features
        node_feats = {'0': batched_graph.ndata['pe'][:, :self.NODE_FEATURE_DIM, None]}
        
        return (batched_graph, node_feats, edge_feats)
    
def eval_gen(batch_size=8,z=12):
    
    in_z = torch.randn((batch_size,z), device='cuda',dtype = torch.float32)
    out = hg(in_z)*31
    out = out.reshape((-1,8,3)).detach().cpu().numpy()
    
    return eval_endpoints(out)
    
    

def eval_endpoints(ep_in): 
    
    ep = ep_in.reshape((-1,8,3))

    v1 = np.arange(ep.shape[1]-1) #vertex 1 of edges in chronological order
    v2 = np.arange(1,ep.shape[1]) #vertex 2 of edges in chronological order

    hLL = np.linalg.norm(ep[:,v1]-ep[:,v2],axis=2)

    hLoc = np.array([0,2,4,6])
    lLoc = np.array([1,3,5])

    return np.mean(hLL[:,hLoc]), np.mean(hLL[:,lLoc])
        

NameError: name 'Dataset' is not defined